# Working with Result Data

This notebook shows you how to:
- Access result data from simulation tools
- List available files for a SQUID
- Download files using the download API
- Work with output curves and arrays
- Save data to files
- Export data for analysis in other tools

## Key Concepts

nanoHUB simulation results can include:
- **Scalar values**: Single numbers (e.g., temperature, voltage)
- **Curve data**: Arrays of x-y pairs for plotting
- **File references**: Links to output files
- **Metadata**: SQUID, revision, date, etc.

This example uses `pntoy`, a simulation tool (simtool=True) that has downloadable files.

## Setup

In [ ]:
# Add parent directory to path for local development
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
from nanohubremote import Session
from nanohubresults import Results
import json

# Initialize session
auth_data = {
    "grant_type": "personal_token",
    "token": "YOUR_TOKEN_HERE"
}
session = Session(auth_data, url="https://nanohub.org/api")
results = Results(session)

print("✓ Connected to nanoHUB API")

## Fetching Sample Data

Let's fetch a result from a simulation tool (simtool) that includes file data. We'll use `pntoy` as an example.

In [ ]:
print("Fetching sample result from pntoy (simulation tool)...")

query = results.query("pntoy", simtool=True) \
    .limit(1)

response = query.execute()

if response.get('results') and len(response['results']) > 0:
    sample_result = response['results'][0]
    squid = sample_result['squid']
    print(f"\n✓ Retrieved result: {squid}")
    
    # Display available fields
    print(f"\nAvailable fields in result:")
    for key in sorted(sample_result.keys()):
        if key != 'squid':
            value = sample_result[key]
            if isinstance(value, dict) and 'xaxis' in value:
                print(f"  • {key}: curve data ({len(value['xaxis'])} points)")
            else:
                print(f"  • {key}: {value}")
else:
    print("No results found!")
    sample_result = None

## Listing Available Files

Before downloading files, you can check what files are available for a specific SQUID using the `get_squid_files()` method. This returns a list of files with their IDs, names, and sizes.

In [ ]:
if sample_result:
    print(f"Fetching file list for SQUID: {squid}\n")
    
    try:
        files_response = results.get_squid_files(squid, simtool=True)
        
        if files_response.get('success'):
            print(f"Response structure:")
            print(f"  • total_files: {files_response.get('total_files', 0)}")
            print(f"  • squid: {files_response.get('squid')}")
            print(f"  • success: {files_response.get('success')}")
            
            print(f"\nFiles available:")
            for i, file_info in enumerate(files_response.get('results', []), 1):
                print(f"\n  File {i}:")
                print(f"    ID: {file_info.get('id')}")
                print(f"    Name: {file_info.get('name')}")
                print(f"    Size: {file_info.get('size'):,} bytes ({file_info.get('size')/1024:.2f} KB)")
            
            # Save the file list
            with open('squid_files_list.json', 'w') as f:
                json.dump(files_response, f, indent=2)
            print(f"\n✓ File list saved to: squid_files_list.json")
        else:
            print("No files found or API returned an error")
            
    except Exception as e:
        print(f"Error retrieving file list: {e}")
else:
    print("No SQUID available to query")

## Downloading Files

Now that we know what files are available, let's download one using the `download()` method.

In [ ]:
if sample_result and files_response.get('results'):
    # Get the first file from the list
    first_file = files_response['results'][0]
    file_name = first_file.get('name')
    
    print(f"Downloading file: {file_name}")
    print(f"  File ID: {first_file.get('id')}")
    print(f"  Size: {first_file.get('size')/1024:.2f} KB\n")
    
    try:
        # Download using the file name
        download_response = results.download(
            tool="pntoy",
            squid=squid,
            file_name=file_name,
            simtool=True
        )
        
        print(f"✓ Download successful!")
        print(f"  Response type: {type(download_response)}")
        
        # Save the downloaded file
        output_filename = f"downloaded_{file_name}"
        
        # If response is JSON (dict), save as JSON
        if isinstance(download_response, dict):
            with open(output_filename, 'w') as f:
                json.dump(download_response, f, indent=2)
            print(f"  Saved as JSON: {output_filename}")
        # If response is binary data (bytes), save as binary
        elif isinstance(download_response, bytes):
            with open(output_filename, 'wb') as f:
                f.write(download_response)
            print(f"  Saved as binary: {output_filename}")
        # If response is string, save as text
        else:
            with open(output_filename, 'w') as f:
                f.write(str(download_response))
            print(f"  Saved as text: {output_filename}")
            
    except Exception as e:
        print(f"✗ Download failed: {e}")
        print(f"  This may happen if the download endpoint is not fully implemented for this tool")
else:
    print("No files available to download")

## Understanding Curve Data

Result data may also include curve data, which is typically structured as:
```python
{
    'xaxis': [x1, x2, x3, ...],
    'yaxis': [y1, y2, y3, ...]
}
```

Let's examine any curve data in our result.

In [ ]:
if sample_result:
    # Find curve fields
    curve_fields = [k for k, v in sample_result.items() 
                    if isinstance(v, dict) and 'xaxis' in v]
    
    if curve_fields:
        print(f"Found {len(curve_fields)} curve field(s):\n")
        
        for field_name in curve_fields[:2]:  # Show first 2 curves
            curve = sample_result[field_name]
            print(f"{field_name}:")
            print(f"  X-axis points: {len(curve.get('xaxis', []))}")
            print(f"  Y-axis points: {len(curve.get('yaxis', []))}")
            print(f"  First few X values: {curve.get('xaxis', [])[:3]}")
            print(f"  First few Y values: {curve.get('yaxis', [])[:3]}\n")
    else:
        print("No curve data found in this result")

## Saving Data to JSON

You can easily save result data to JSON files for later analysis or sharing.

In [ ]:
if sample_result:
    # Save complete result
    output_file = "pntoy_result.json"
    
    with open(output_file, "w") as f:
        json.dump(sample_result, f, indent=2)
    
    print(f"✓ Complete result saved to: {output_file}")

## Converting Curve Data to CSV

For curve data, CSV format is often more convenient for analysis tools like Excel.

In [ ]:
import csv

if sample_result:
    # Find first curve field
    curve_fields = [k for k, v in sample_result.items() 
                    if isinstance(v, dict) and 'xaxis' in v]
    
    if curve_fields:
        field_name = curve_fields[0]
        curve = sample_result[field_name]
        csv_file = f"pntoy_{field_name.replace('.', '_')}.csv"
        
        with open(csv_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['X', 'Y'])  # Header
            
            # Write data rows
            for x, y in zip(curve['xaxis'], curve['yaxis']):
                writer.writerow([x, y])
        
        print(f"✓ Curve data saved to CSV: {csv_file}")
        print(f"  Format: {len(curve['xaxis'])} rows × 2 columns")
    else:
        print("No curve data to export")

## Summary

In this notebook, you learned:
1. ✓ How to list available files for a SQUID using `get_squid_files()`
2. ✓ How to download files using the `download()` method
3. ✓ How to access curve and scalar data from results
4. ✓ How to save data in JSON format
5. ✓ How to export curve data to CSV for analysis

## Best Practices

- **Check files first**: Use `get_squid_files()` to see what's available before attempting downloads
- **Use JSON for complete data**: Preserves all structure and metadata
- **Use CSV for curve data**: Better for plotting and analysis tools
- **Include metadata**: Always save SQUID and input parameters with results
- **Simulation tools only**: File listing and download only work for simulation tools (simtool=True)

## API Methods for File Access

The library provides two complementary methods:

1. **`get_squid_files(squid, simtool=True)`**: List all files associated with a simulation run
   - Returns file IDs, names, and sizes
   - Use this to discover what's available
   - Only works for simulation tools

2. **`download(tool, squid, field=None, file_name=None, simtool=False)`**: Download specific files or data
   - Use `field` for specific output fields
   - Use `file_name` for files discovered via `get_squid_files()`
   - Set `simtool=True` when downloading from simulation tools

## Next Steps

- `04_pagination.ipynb` - Learn how to efficiently iterate over large result sets